<a href="https://colab.research.google.com/github/Kiyarash00/KD_Empire/blob/main/Premier_League_Sentiment_Recognizer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
import pickle
import os
import string

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
pip install kaggle --quiet

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d wjia26/epl-teams-twitter-sentiment-dataset

Dataset URL: https://www.kaggle.com/datasets/wjia26/epl-teams-twitter-sentiment-dataset
License(s): GPL-2.0
 54% 52.0M/97.0M [00:00<00:00, 532MB/s]
100% 97.0M/97.0M [00:00<00:00, 511MB/s]


In [ ]:
!unzip epl-teams-twitter-sentiment-dataset.zip

Archive:  epl-teams-twitter-sentiment-dataset.zip
  inflating: 2020-07-09 till 2020-09-19.csv  
  inflating: 2020-09-20 till 2020-10-13.csv  


In [ ]:
#Reading Files
df1 = pd.read_csv("/content/2020-07-09 till 2020-09-19.csv")
df2 = pd.read_csv("/content/2020-09-20 till 2020-10-13.csv")

In [ ]:
df1

,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1
0,2020-07-09 11:34:08,Liverpool,697325,594,Liverpool FC,Liverpool,0,LivEchoLFC,#liverpoolfc OR #YNWA OR #LFC,This is a strange claim #LFC https://t.co/U1...,1281189890936504320,Liverpool FC News,-0.2023,EPL,Liverpool
1,2020-07-09 11:33:52,Liverpool,2348,2907,Liverpool FC,Huyton,65,ged11lfc,#liverpoolfc OR #YNWA OR #LFC,RT @TheKopiteOFF: 😱| #LFC have won 30 of their...,1281189827468279808,Ged Smith,0.8316,EPL,Liverpool
2,2020-07-09 11:33:52,Liverpool,465,177,Liverpool FC,"Accra, Ghana",32,calvin_winzy,#liverpoolfc OR #YNWA OR #LFC,RT @SkySportsPL: 🔴 - #LFC secure first away wi...,1281189824465174528,DADA BA♐,0.8793,EPL,Liverpool
3,2020-07-09 11:33:50,Liverpool,334,1524,Liverpool FC,NaN,0,Rurs1,#liverpoolfc OR #YNWA OR #LFC,Outrageous... Poor auld Martin Tyler has to ju...,1281189816890245121,Ruairi Nolan,-0.6808,EPL,Liverpool
4,2020-07-09 11:33:31,Liverpool,760,1264,Liverpool FC,"Kildare, Ireland 🇮🇪",176,LSHighVoltage,#liverpoolfc OR #YNWA OR #LFC,RT @TheRedmenTV: ✅ Won the league ✅ Fastest te...,1281189739618611200,🇮🇪 David D'Arcy 🇮🇪,0.8176,EPL,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799235,2020-08-11 17:16:08,Southampton,947,1528,Southampton FC,"England, United Kingdom, 🇬🇧",0,LoopilouI,#SouthamptonFC OR #Southampton,I honestly don't understand &amp; think it's d...,1293234760299483136,SunThruTheRain🌈,-0.4391,EPL,Southampton
799236,2020-08-11 17:15:17,Southampton,1887,1675,Southampton FC,"Southampton, England",0,FieldPalmer,#SouthamptonFC OR #Southampton,Fantastically positioned in highly sought afte...,1293234544187969538,Field Palmer Estate Agents,0.6428,EPL,Southampton
799237,2020-08-11 17:12:32,Southampton,934,1468,Southampton FC,Sheffield,0,johnjpkennedy,#SouthamptonFC OR #Southampton,@SkySportsNews Not a bad player tbh hope #Sout...,1293233854539595778,John Kennedy,0.8248,EPL,Southampton
799238,2020-08-11 17:07:20,Southampton,84753,2118,Southampton FC,NaN,12,NicoSchira,#SouthamptonFC OR #Southampton,Now is official! Pierre-Emile #Højbjerg to #To...,1293232545052712965,Nicolò Schira,0.0000,EPL,Southampton


In [ ]:
df2

,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1
0,2020-09-23 18:39:34,CrystalPalace,66,495,Crystal Palace FC,NaN,0,tw_footballtalk,#CrystalPalace OR #CPFC,Onto Week 2 of Premier League Stars of the Wee...,1308838436233711616,TW,0.0000,EPL,CrystalPalace
1,2020-09-23 18:39:18,CrystalPalace,0,0,Crystal Palace FC,NaN,0,crystalpalaced1,#CrystalPalace OR #CPFC,Our palace team will put our hearts out for ou...,1308838367254269954,crystalpalacedan,0.1531,EPL,CrystalPalace
2,2020-09-23 18:28:15,CrystalPalace,4292,3437,Crystal Palace FC,London,0,palaceupdates4u,#CrystalPalace OR #CPFC,Chelsea v Palace kick-off time changed and TV ...,1308835588229996545,Crystal Palace News,0.0000,EPL,CrystalPalace
3,2020-09-23 18:28:15,CrystalPalace,4292,3437,Crystal Palace FC,London,0,palaceupdates4u,#CrystalPalace OR #CPFC,Academy redevelopment update: Exclusive footag...,1308835585902116867,Crystal Palace News,0.1280,EPL,CrystalPalace
4,2020-09-23 18:27:39,CrystalPalace,0,0,Crystal Palace FC,NaN,0,crystalpalaced1,#CrystalPalace OR #CPFC,@ChelseaFC I still have no entire clue why us ...,1308835436652105728,crystalpalacedan,-0.1027,EPL,CrystalPalace
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211337,2020-09-30 04:45:00,Everton,10962,12153,Everton FC,United Kingdom,0,bash_the_bookie,#Everton OR #EFC,"BTTS, Each Team Over 3 Corners &amp; Each Team...",1311165122849669120,Inplay Challenges,0.0000,EPL,Everton
211338,2020-09-30 04:35:12,Everton,3439,3277,Everton FC,Litherland,0,KevBlank9,#Everton OR #EFC,Matchday ! 💙 #UpTheToffees #EFC #COYB,1311162658687725569,Kev,0.0000,EPL,Everton
211339,2020-09-30 04:28:12,Everton,0,0,Everton FC,NaN,0,futbol_boss,#Everton OR #EFC,First post!! Come ride along and enjoy the bea...,1311160896962678785,Fútbol Boss,0.8264,EPL,Everton
211340,2020-09-30 04:12:52,Everton,31,312,Everton FC,"England, United Kingdom",1,__RobMatthews__,#Everton OR #EFC,New home on the way..... #EFC #Everton #Spirit...,1311157039389962247,Rob Matthews,0.0000,EPL,Everton


In [ ]:
#Unifying The Datasets
df = pd.concat([df1, df2], axis=0, ignore_index=True)

In [ ]:
df

,created_at,file_name,followers,friends,group_name,location,retweet_count,screenname,search_query,text,twitter_id,username,polarity,partition_0,partition_1
0,2020-07-09 11:34:08,Liverpool,697325,594,Liverpool FC,Liverpool,0,LivEchoLFC,#liverpoolfc OR #YNWA OR #LFC,This is a strange claim #LFC https://t.co/U1...,1281189890936504320,Liverpool FC News,-0.2023,EPL,Liverpool
1,2020-07-09 11:33:52,Liverpool,2348,2907,Liverpool FC,Huyton,65,ged11lfc,#liverpoolfc OR #YNWA OR #LFC,RT @TheKopiteOFF: 😱| #LFC have won 30 of their...,1281189827468279808,Ged Smith,0.8316,EPL,Liverpool
2,2020-07-09 11:33:52,Liverpool,465,177,Liverpool FC,"Accra, Ghana",32,calvin_winzy,#liverpoolfc OR #YNWA OR #LFC,RT @SkySportsPL: 🔴 - #LFC secure first away wi...,1281189824465174528,DADA BA♐,0.8793,EPL,Liverpool
3,2020-07-09 11:33:50,Liverpool,334,1524,Liverpool FC,NaN,0,Rurs1,#liverpoolfc OR #YNWA OR #LFC,Outrageous... Poor auld Martin Tyler has to ju...,1281189816890245121,Ruairi Nolan,-0.6808,EPL,Liverpool
4,2020-07-09 11:33:31,Liverpool,760,1264,Liverpool FC,"Kildare, Ireland 🇮🇪",176,LSHighVoltage,#liverpoolfc OR #YNWA OR #LFC,RT @TheRedmenTV: ✅ Won the league ✅ Fastest te...,1281189739618611200,🇮🇪 David D'Arcy 🇮🇪,0.8176,EPL,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010577,2020-09-30 04:45:00,Everton,10962,12153,Everton FC,United Kingdom,0,bash_the_bookie,#Everton OR #EFC,"BTTS, Each Team Over 3 Corners &amp; Each Team...",1311165122849669120,Inplay Challenges,0.0000,EPL,Everton
1010578,2020-09-30 04:35:12,Everton,3439,3277,Everton FC,Litherland,0,KevBlank9,#Everton OR #EFC,Matchday ! 💙 #UpTheToffees #EFC #COYB,1311162658687725569,Kev,0.0000,EPL,Everton
1010579,2020-09-30 04:28:12,Everton,0,0,Everton FC,NaN,0,futbol_boss,#Everton OR #EFC,First post!! Come ride along and enjoy the bea...,1311160896962678785,Fútbol Boss,0.8264,EPL,Everton
1010580,2020-09-30 04:12:52,Everton,31,312,Everton FC,"England, United Kingdom",1,__RobMatthews__,#Everton OR #EFC,New home on the way..... #EFC #Everton #Spirit...,1311157039389962247,Rob Matthews,0.0000,EPL,Everton


In [ ]:
#Checking df Info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010582 entries, 0 to 1010581
Data columns (total 15 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   created_at     1010582 non-null  object 
 1   file_name      1010582 non-null  object 
 2   followers      1010582 non-null  int64  
 3   friends        1010582 non-null  int64  
 4   group_name     1010582 non-null  object 
 5   location       710490 non-null   object 
 6   retweet_count  1010582 non-null  int64  
 7   screenname     1010582 non-null  object 
 8   search_query   1010582 non-null  object 
 9   text           1010582 non-null  object 
 10  twitter_id     1010582 non-null  int64  
 11  username       1010566 non-null  object 
 12  polarity       1010582 non-null  float64
 13  partition_0    1010582 non-null  object 
 14  partition_1    1010582 non-null  object 
dtypes: float64(1), int64(4), object(10)
memory usage: 115.7+ MB
None


In [ ]:
#Finding Useful Values
#print(df['created_at'].value_counts())
#print(df['file_name'].value_counts())
#print(df['followers'].value_counts())
#print(df['friends'].value_counts())
#print(df['group_name'].value_counts())
#print(df['location'].value_counts())
#print(df['retweet_count'].value_counts())
#print(df['screenname'].value_counts())
#print(df['search_query'].value_counts())
print(df['text'].value_counts())
#print(df['twitter_id'].value_counts())
#print(df['username'].value_counts())
print(df['polarity'].value_counts())
#print(df['partition_0'].value_counts())
#print(df['partition_1'].value_counts())

text
RT @MDfineart: Ship and ice creams at Mayflower Park Southampton ART https://t.co/h49rntGKmw #ships #Southampton #landscape #marine #art ht…                                                                                              2486
RT @PFA: 💙 Chelsea’s @toniruediger funded 60,000 facemasks to help keep market traders in Sierra Leone safe. He also showed his support for…                                                                                              2249
RT @footballisfixed: #EPL Table without ref 'errors' #Liverpool 81 #ManCity 71 #Arsenal 65 #Wolves 54 #Leicester 52 #Chelsea 51 #Spurs 51 #…                                                                                              2182
RT @CazooUK: Win an @Everton shirt with Cazoo 💙  Simply retweet this post and follow our page!   Five prizes up for grabs - winners can cho…                                                                                              1331
RT @SpursOfficial: FULL-TIME: Our final

In [ ]:
#Dropping Unnecessary Columns
df.drop(["followers", "friends", "group_name", "location", "retweet_count", "screenname", "twitter_id", "username", "partition_0", "partition_1","created_at", "file_name", "search_query"], axis=1, inplace = True)

In [ ]:
df

,text,polarity
0,This is a strange claim #LFC https://t.co/U1...,-0.2023
1,RT @TheKopiteOFF: 😱| #LFC have won 30 of their...,0.8316
2,RT @SkySportsPL: 🔴 - #LFC secure first away wi...,0.8793
3,Outrageous... Poor auld Martin Tyler has to ju...,-0.6808
4,RT @TheRedmenTV: ✅ Won the league ✅ Fastest te...,0.8176
...,...,...
1010577,"BTTS, Each Team Over 3 Corners &amp; Each Team...",0.0000
1010578,Matchday ! 💙 #UpTheToffees #EFC #COYB,0.0000
1010579,First post!! Come ride along and enjoy the bea...,0.8264
1010580,New home on the way..... #EFC #Everton #Spirit...,0.0000


In [ ]:
# data
X = df['text'].astype(str).values
y = df['polarity'].astype(float).values

# splits: 80/10/10
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)
X_val, X_test, y_val, y_test    = train_test_split(X_tmp, y_tmp, test_size=0.50, random_state=42, shuffle=True)

# check
print("lens:", X_train.shape[0], y_train.shape[0], X_val.shape[0], y_val.shape[0], X_test.shape[0], y_test.shape[0])

#preparing ignore_list and tokenizer for TfidfVectorizer
def simple_preprocessor(text: str) -> str:
    if text is None:
        return ""
    text = text.lower()
    # remove URLs
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    # remove user mentions like @user
    text = re.sub(r"@\w+", " ", text)
    # collapse whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

def simple_tokenizer(text: str):
  tokens = []
  for tok in text.split():
      tok = tok.strip(string.punctuation + "“”‘’\"'`")
      if tok:
          tokens.append(tok)
  return tokens

# vectorize
vectorizer = TfidfVectorizer(preprocessor=simple_preprocessor, tokenizer=simple_tokenizer, ngram_range=(1,2), lowercase=True, min_df=2, max_features=20000, sublinear_tf=True)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf   = vectorizer.transform(X_val)
X_test_tfidf  = vectorizer.transform(X_test)

print("tfidf shapes:", X_train_tfidf.shape[0], X_val_tfidf.shape[0], X_test_tfidf.shape[0])

# model
model = lgb.LGBMRegressor(n_estimators=2500, learning_rate=0.08, num_leaves=31, random_state=42, n_jobs=-1)

# fit
model.fit(
    X_train_tfidf, y_train,
    eval_set=[(X_val_tfidf, y_val)],
    eval_metric='l1',
    callbacks=[early_stopping(stopping_rounds=100), log_evaluation(period=100)]
)

# evaluation
y_pred = model.predict(X_test_tfidf)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

lens: 808465 808465 101058 101058 101059 101059
tfidf shapes: 808465 101058 101059
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 290.146892 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 840321
[LightGBM] [Info] Number of data points in the train set: 808465, number of used features: 19999
[LightGBM] [Info] Start training from score 0.216872
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 0.169562	valid_0's l2: 0.0582632
[200]	valid_0's l1: 0.14159	valid_0's l2: 0.0445309
[300]	valid_0's l1: 0.12699	valid_0's l2: 0.0385103
[400]	valid_0's l1: 0.118032	valid_0's l2: 0.0352355
[500]	valid_0's l1: 0.112182	valid_0's l2: 0.0332328
[600]	valid_0's l1: 0.107945	valid_0's l2: 0.031886
[700]	valid_0's l1: 0.104826	valid_0's l2: 0.0309241
[800]	valid_0's l1: 0.10247	valid_0's l2: 0.0302256
[900]	valid_0's l1: 0

In [34]:
# saving vectorizer and model
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

with open("lgbm_regressor.pkl", "wb") as f:
  pickle.dump(model, f)

In [35]:
#loading vectorizer and model
with open("tfidf_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)

with open("lgbm_regressor.pkl", "rb") as f:
    model = pickle.load(f)

In [36]:
neg_threshold = -0.01
pos_threshold = 0.01

# Paths to try for a local word list (one word per line)
WORDLIST_PATHS = [
    "english_words.txt",
    "words.txt",
    "/usr/share/dict/words",
    "/usr/dict/words"
]

def load_wordset():
    for p in WORDLIST_PATHS:
        if os.path.exists(p):
            try:
                with open(p, "r", encoding="utf8", errors="ignore") as f:
                    words = {line.strip().lower() for line in f if line.strip()}
                if words:
                    return words
            except Exception:
                continue
    return None

WORDSET = load_wordset()  # may be None

def is_unrelatable(text: str) -> bool:
    if text is None:
        return True

    text_stripped = text.strip()
    if text_stripped == "":
        return True

    text_lower = text_stripped.lower()

    # quick reject: only digits/punctuation
    if not any(ch.isalpha() for ch in text_lower):
        return True

    # split to tokens by whitespace and strip punctuation from ends
    tokens = []
    for tok in text_lower.split():
        tok = tok.strip(string.punctuation + "“”‘’\"'`")
        if tok:
            tokens.append(tok)

    if not tokens:
        return True

    # remove tokens that are purely punctuation or single digits
    tokens = [t for t in tokens if any(ch.isalpha() for ch in t)]
    if not tokens:
        return True

    # If we have a wordlist, use it to compute real-word ratio
    if WORDSET is not None:
        recognized = sum(1 for t in tokens if t in WORDSET)
        real_ratio = recognized / len(tokens)
        if real_ratio < 0.5:
            return True
        return False

    # Fallback heuristics when no wordlist available
    vowels = set("aeiou")
    tokens_with_vowel = sum(1 for t in tokens if any(ch in vowels for ch in t))

    def has_long_consonant_run(tok, limit=4):
        run = 0
        for ch in tok:
            if ch.isalpha() and ch not in vowels:
                run += 1
                if run >= limit:
                    return True
            else:
                run = 0
        return False

    consonant_run_count = sum(1 for t in tokens if has_long_consonant_run(t))
    avg_len = sum(len(t) for t in tokens) / len(tokens)

    # Heuristic decision thresholds (tunable)
    MIN_VOWEL_RATIO = 0.5
    MAX_CONSONANT_RUNS = max(1, len(tokens) // 2)
    MIN_AVG_LEN = 2.2

    if (tokens_with_vowel / len(tokens)) < MIN_VOWEL_RATIO:
        return True
    if consonant_run_count > MAX_CONSONANT_RUNS:
        return True
    if avg_len < MIN_AVG_LEN and len(tokens) <= 3:
        return True

    return False

def predict_sentiment(text: str):
    arr = np.array([str(text)])
    X = vectorizer.transform(arr)
    score = float(model.predict(X)[0])

    if score < neg_threshold:
        label = "negative"
    elif score > pos_threshold:
        label = "positive"
    else:
        label = "neutral"

    return score, label

if __name__ == "__main__":
    text = input("Enter your sentence: ").strip()
    if is_unrelatable(text):
        print("The input is unrelatable.")
    else:
        score, label = predict_sentiment(text)
        print(f"Polarity score: {score:.4f}")
        print(f"Sentiment label: {label}")

Enter your sentence: that was a clean tackle by Maguire
Polarity score: 0.2950
Sentiment label: positive
